In [1]:
import numpy
from numpy import zeros
from numpy import logical_and
from osgeo import gdal

In [2]:
def reclassify(infile, year):

    classification_values = [0, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
                             2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]  # The interval values to classify
    classification_output_values = [0]*len(classification_values)
    index = classification_values.index(year)
    classification_output_values[index] = year

    in_file = infile

    ds = gdal.Open(in_file)
    band = ds.GetRasterBand(1)

    block_sizes = band.GetBlockSize()
    x_block_size = block_sizes[0]
    y_block_size = block_sizes[1]

    xsize = band.XSize
    ysize = band.YSize

    max_value = band.GetMaximum()
    min_value = band.GetMinimum()

    if max_value == None or min_value == None:
        stats = band.GetStatistics(0, 1)
        max_value = stats[1]
        min_value = stats[0]

    NDV, xsize, ysize, GeoT, Projection, DataType, driver = GetGeoInfo(infile)

    if DataType == 'Float32':
        DataType = gdal.GDT_Float32

    NewFileName = str(infile.split(".")[0])+'_'+str(year)+'.tif'
    # Set up the dataset
    dst_ds = driver.Create(NewFileName, xsize, ysize, 1, DataType)
    # the '1' is for band 1.
    dst_ds.SetGeoTransform(GeoT)
    dst_ds.SetProjection(Projection.ExportToWkt())

    for i in range(0, ysize, y_block_size):
        if i + y_block_size < ysize:
            rows = y_block_size
        else:
            rows = ysize - i
        for j in range(0, xsize, x_block_size):
            if j + x_block_size < xsize:
                cols = x_block_size
            else:
                cols = xsize - j

            data = band.ReadAsArray(j, i, cols, rows)
            r = zeros((rows, cols), numpy.uint8)

            for k in range(len(classification_values) - 1):
                if classification_values[k] <= max_value and (classification_values[k + 1] > min_value):
                    r = r + classification_output_values[k] * logical_and(
                        data >= classification_values[k], data < classification_values[k + 1])
            if classification_values[k + 1] <= max_value:
                r = r + \
                    classification_output_values[k+1] * \
                    (data >= classification_values[k + 1])

            dst_ds.GetRasterBand(1).WriteArray(r, j, i)

    dst_ds = None

In [3]:
from gdalconst import *
from osgeo import osr

# Function to read the original file's projection:


def GetGeoInfo(FileName):
    SourceDS = gdal.Open(FileName, GA_ReadOnly)
    NDV = SourceDS.GetRasterBand(1).GetNoDataValue()
    xsize = SourceDS.RasterXSize
    ysize = SourceDS.RasterYSize
    GeoT = SourceDS.GetGeoTransform()
    Projection = osr.SpatialReference()
    Projection.ImportFromWkt(SourceDS.GetProjectionRef())
    DataType = SourceDS.GetRasterBand(1).DataType
    DataType = gdal.GetDataTypeName(DataType)
    driver = gdal.GetDriverByName('GTiff')

    return NDV, xsize, ysize, GeoT, Projection, DataType, driver

In [6]:
year_list = range(2001, 2019, 1)
for year in year_list:
    reclassify('SUAV_third.tif', year)

In [7]:
def sum_year(year):

    list_files = ['SUAV_first_'+str(year)+'.tif', 'SUAV_second_'+str(
        year)+'.tif', 'SUAV_third_'+str(year)+'.tif']

    ds_first = gdal.Open(list_files[0])
    band_first = ds_first.GetRasterBand(1).ReadAsArray()

    ds_second = gdal.Open(list_files[1])
    band_second = ds_second.GetRasterBand(1).ReadAsArray()

    ds_third = gdal.Open(list_files[2])
    band_third = ds_third.GetRasterBand(1).ReadAsArray()

    sum_band = band_first + band_second + band_third

    NDV, xsize, ysize, GeoT, Projection, DataType, driver = GetGeoInfo(
        list_files[0])

    if DataType == 'Float32':
        DataType = gdal.GDT_Float32
    NewFileName = 'SUAVSUM_'+str(year)+'.tif'

    # Set up the dataset
    DataSet = driver.Create(NewFileName, xsize, ysize, 1, DataType)
    # the '1' is for band 1.
    DataSet.SetGeoTransform(GeoT)
    DataSet.SetProjection(Projection.ExportToWkt())
    # Write the array
    DataSet.GetRasterBand(1).WriteArray(sum_band)

    return NewFileName

In [8]:
year_list = range(2001, 2019, 1)

for year in year_list:
    sum_year(year)